# SRT Lab: Intro to Digital Signal Processing for Radio Astronomy

## Introduction

The 21cm line produced by neutral hydrogen in interstellar space provides radio astronomers with a very useful probe for studying the differential rotation of spiral galaxies. By observing hydrogen lines at different points along the Galactic plane one can show that the angular velocity increases as you look at points closer to the Galactic center. The purpose of this experiment is to create a rotational curve for the Milky Way Galaxy using 21-cm spectral lines observed with a small radio telescope. The sample observations for this experiment will be made using the small radio telescope located at the Haystack Observatory. The rotational curve will be created by plotting the maximum velocity observed along each line of sight versus the distance of this point from the Galactic center. 

Hydrogen is the most abundant element in the cosmos; it makes up 80% of the universe’s mass.  Therefore, it is no surprise that one of the most significant spectral lines in radio astronomy is the 21-cm hydrogen line. In interstellar space, gas is extremely cold. Therefore, hydrogen atoms in the interstellar medium are at such low temperatures (~100 K) that they are in the ground electronic state. This means that the electron is as close to the nucleus as it can get, and it has the lowest allowed energy. Radio spectral lines arise from changes between one energy level to another.

A neutral hydrogen atom consists of one proton and one electron, in orbit around the nucleus. Both the proton and the electron spin about their individual axes, but they do not spin in just one direction. They can spin in the same direction (parallel) or in opposite directions (anti-parallel). The energy carried by the atom in the parallel spin is greater than the energy it has in the anti-parallel spin. Therefore, when the spin state flips from parallel to anti parallel, energy (in the form of a low energy photon) is emitted at a radio wavelength of 21-cm. This 21-cm radio spectral line corresponds to a frequency of 1.420 GHz.

The first person to predict this 21-cm line for neutral hydrogen was H. C. van de Hulst in 1944. However, it was not until 1951 that a Harvard team created the necessary equipment, and the first detection of this spectral line was made. 

One reason this discovery is so significant is because hydrogen radiation is not impeded by interstellar dust. Optical observations of the Galaxy are limited due to the interstellar dust, which does not allow the penetration of light waves. However, this problem does not arise when making radio measurements of the HI region. Radiation from this region can be detected anywhere in our Galaxy.

Measurements of the HI region of the Galaxy can be used in various calculations. For example, observations of the 21-cm line can be used to create the rotation curve for our Milky Way Galaxy. If hydrogen atoms are distributed uniformly throughout the Galaxy, a 21-cm line will be detected from all points along the line of sight of our telescope. The only difference will be that all of these spectra will have different Doppler shifts. Once the rotation curve for the Galaxy is known, it can be used to find the distances to various objects. By knowing the Doppler shift of a body, its angular velocity can be calculated. Combining this angular velocity and the plot of the rotation curve, the distance to a certain object can be inferred. Using measurements of the HI region, the mass of the Galaxy can also be determined.

**In this lab, we will use the SRT to observe the neutral hydrogen line and explore the nature of radio signals and their analysis for astronomical purposes.**

In [ ]:
from IPython.display import HTML
import digital_rf
from astropy.io import fits
import math
import pandas as pd
import cmath
import plotly.graph_objects as go
import sk_dsp_comm.sigsys as ss
import scipy.signal as signal
from IPython.display import Image, SVG
%pylab inline
from ipywidgets import interact, interactive
import os
from astrocalc import *
from dataprocessing import SRTData
pylab.rcParams['savefig.dpi'] = 100 # default 72
%config InlineBackend.figure_formats=['svg'] # SVG inline viewing

## I. Fourier Transforms

The Fourier transform is an incredibly important tool used across a wide variety of fields in science, math, and engineering. It is of particular importance to radio astronomers because it is a key component of digital signal processing, data processing, and instrumentation, in addition to being a cornerstone of interferometry and aperture synethsis. 
The continuous Fourier transform is a reversible, linear transform defined for a complex, integrable function f(x) as:    
$ F(s) = \int_{-\infty}^{\infty}f(x)e^{-i 2 \pi s x}dx$      
The Fourier transform of the waveform $f(t)$ expressed as a time-domain signal is the spectrum $F(\nu)$ expressed as the frequency domain signal, where t is in seconds and $\nu$ is in Hz.
Complex exponentials, or complex numbers where the real and imaginary components are sinusoids, are key to the transform. They can be expressed using Euler's formula, $e^{i \phi} = \cos{\phi} + i\sin{\phi}$, and they form a set that is complete and orthogonal. This property of complex exponentials makes Fourier transforms highly useful in fields ranging from radio propagation to quantum mechanics, because it means that they can represent any piecewise continuous function and minimize the least-square error between the function and its representation.   
While the continuous Fourier transform converts a time-domain signal of infinite duration into a continuous spectrum composed of an infinite number of sinusoids, in practice we tend to deal with signals that are discretely sampled (usually at constant intervals) and are periodic or of finite duration. This is where the discrete Fourier transform (DFT) comes in, because only a finite number of sinusoids are needed. Consider the equation for the DFT of a signal x of length N:
$X[k] = \sum_{n=0}^{N-1} x[n] e^{-i 2 \pi \frac{k n}{N}}$   
The DFT of an N-point input in the time domain is an N-point frequency spectrum, with Fourier frequencies k randing from $-(\frac{N}{2}-1)$ to $\frac{N}{2}$ which pass through the 0 frequency which is also known as the DC component. Each bin number represents the number of sinusoidal periods present in the time domain and can be described by $X_k=A_k e^{i \phi_k}$, where $A_k$ and $\phi_k$ are the respective amplitudes and phases of each sinusoid.    
Usually, the DFT is computed by an algorithm known as the fast Fourier transform (FFT), which is ubiquitous in signal processing and modern electronics. Try using the code below to experiment with the fast Fourier transform. 

In [ ]:
def fft_approx(x,t,Nfft):
    fs = 1/(t[1] - t[0])
    t0 = (t[-1]+t[0])/2 # time delay at center
    N0 = len(t)/2 # FFT center in samples
    f = np.arange(-1/2,1/2,1/Nfft)
    w, X = signal.freqz(x,1,2*pi*f)
    X /= fs # account for dt = 1/fs in integral
    X *= exp(-1j*2*pi*f*fs*t0)# time interval correction
    X *= exp(1j*2*pi*f*N0)# FFT time interval is [0,Nfft-1]
    F = f*fs
    return F, X

def plot_fft(D1,D2,W1,W2,harmonics1,harmonics2):
    t = arange(-5,5,.01)
    x_rect = ss.rect(t-D1,W1)
    x_tri = ss.tri(t-D2,W2)
    for h in range(harmonics1):
        x_rect = x_rect + ss.rect(t-D1,(h+1)*W1)
    for h in range(harmonics2):
        x_tri = x_tri + ss.tri(t-D2,(h+1)*W2)
    #plot square wave
    subplot(221)
    plot(t,x_rect)
    grid()
    xlabel(r'Time (s)')
    ylabel(r'$\Pi((t-3)/2)$')
    #plot triangular wave
    subplot(222)
    plot(t,x_tri)
    grid()
    xlabel(r'Time (s)')
    ylabel(r'$\Lambda((t+2)/1.5)$')
    tight_layout()
    # fast fourier transform of square wave
    f,X0 = fft_approx(x_rect,t,4096)
    subplot(223)
    plot(f,abs(X0)) #plot magnitude
    #plot(f,angle(X0))  #plot argument
    grid()
    xlim([-10,10])
    title(r'Approx. Spectrum Magnitude')
    xlabel(r'Frequency (Hz)')
    ylabel(r'$|X_0(\Pi)|$')
    tight_layout()
    # fast fourier transform of triangular wave
    f,X0 = fft_approx(x_tri,t,4096)
    subplot(224)
    plot(f,abs(X0))  #plot magnitude
    #plot(f,angle(X0))  #plot argument
    grid()
    xlim([-10,10])
    title(r'Approx. Spectrum Magnitude')
    xlabel(r'Frequency (Hz)')
    ylabel(r'$|X_0(\Lambda)|$')
    tight_layout()

interactive_plot = interactive(plot_fft,D1 = (-3,3,.5), D2 = (-3,3,.5), 
                               W1 = (0.5,2,.25), W2 = (0.5,2,.25),
                              harmonics1=(0,10,1),harmonics2=(0,10,1));
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

## II. Collecting Data

![SRT Dash interface](imgs/srtui.png)

1. Start up the SRT Dash interface
2. Set the center frequency to 1420.4 MHz (the hydrogen line), with a bandwidth of 2 MHz. Navigate to the "routine" menu and click "start" to begin recording data; let it run for some arbitrary amount of time before clicking "stop" to end the recording.
3. Enter the directory of your recorded data channel in the cell below

## Read in .fits Data

In [ ]:
#Specify filepath to fits data
datadir = 'srt-galactic-test/g00.fits'
hdul = fits.open(datadir)
data = hdul[0].data/len(hdul)
f_s = 2e6
n_spec = len(data)
for i in range(1,len(hdul)):
    data = data + hdul[i].data/len(hdul)
freqs = np.fft.fftfreq(n_spec,1./f_s)
freqs = np.fft.fftshift(freqs)*1e-6+1420.4
srtdata = pd.DataFrame({'value':data,'frequency':freqs})

## III. Power Spectra

The power spectrum,defined as $\overline{F(s)}F(s)=\lvert F(s) \rvert^2$, is a very useful quantity in astronomy. It preserves no information from the original function. By Rayleigh's Theorem, we know that signal energies are equal in the frequency and time domains because the integral of the power spectrum equals the integral of the squared modulus of the function:   
$\int_{-\infty}^{\infty}\lvert f(x) \rvert^2 dx = \int_{-\infty}^{\infty} \lvert F(s) \rvert^2 ds$    
Use the code below to plot a power spectrum of your SRT data. How does the spectrum change when you change the bin size of the fast fourier transform?

### For .fits:

In [ ]:
plot(srtdata['frequency'],srtdata['value'])
xlabel('Power')
ylabel('Frequency (MHz)')
title('Integrated Power Spectrum')

## IV. Spectrograms

A spectrogram is a three-dimensional plot of signal amplitude versus time and frequency, usually depicted in the time-frequency plane with the third dimension, amplitude, represented by color. It is created by taking chunks of a signal, performing a DFT on each chunk, and then plotting the resulting series of spectra versus time. A spectrogram is a visual way of representing the signal strength, or “loudness”, of a signal over time at various frequencies present in a particular waveform. Use the code below to plot a spectrogram of your SRT data. Try going into the .config file of the srt software and changing the bin size of the Fast Fourier Transform, then collecting and plotting new data; how does the spectrogram change? Can you see the hydrogen line emission?

## For .fits:

In [ ]:
samps = np.row_stack([i.data for i in hdul])
freq_mat,s_mat = np.meshgrid(srtdata['frequency'],np.arange(len(hdul)))
print(freq_mat.shape,s_mat.shape)
plt.pcolormesh(s_mat, freq_mat, np.fft.fftshift(samps,axes=1))
cb = plt.colorbar()
plt.ylabel('Frequency (MHz)')
plt.xlabel('Sample Number')
plt.title("Spectrogram")
cb.set_label("power")

## V. Using the Doppler Effect to Map the Milky Way

Our observations of the Neutral Hydrogen in the galaxy were made around its rest frequency of 1420.4 MHz. However, we observe it at different frequencies since it is moving relative to us in the galaxy.  Using the Doppler effect, we can derive the following formula:


$\frac{\lambda_e - \lambda_o}{\lambda_e} = \frac{\Delta \lambda}{\lambda} = \frac{v}{c}$   


Where $\lambda_e$ is the wavelength emitted, $\lambda_o$ is the wavelength observed, and c is the speed of light (300,000 km/s). To convert from frequency to wavelength, $c=\lambda f$ to get:  


$\frac{f_e-f_o}{f_e} c = v$   


Where $f_e$ is the frequency emitted and $f_o$ is the frequency observed. We know the frequency emitted by the hydrogen line is roughly 1420.4 MHz, so we can use $f_e=1420.4$, which yields $v = \frac{1420.4-f_o}{1420.4} c$.       


However, we aren't done yet; we have to take into account the shift observed both due to the rotation of the Earth and the motion of the sun around the galactic center. To fully account for these factors, it is also necessary to subtract the velocity relative to the local standard of rest ($v_{lsr}$) from this calculation, to get the following:  


$v = \frac{1420.4-f_o}{1420.4} c - v_{lsr}$        


The $v_{lsr}$ can be found if we know latitude, longitude, altitude, azimuth, and the time of the observation. Most of this information, with the exception of latitude and longitude, is already stored in the .fits header of your data, so all you need to enter is the latitude and longitude of your location in the cell below in order to calculate $v_{lsr}$. Using this, we can convert our power vs frequency data to power vs relative velocity. Use the code below to convert your observed frequencies to relative velocities, and then re-plot the spectrogram, this time using relative velocity instead of frequency as the y axis.

In [ ]:
#calculate v_lsr
#Enter your Latitude and Longitude here:
lat = 42.5 #at Haystack
lon = 71.5 #at Haystack
#-------------------------------------
date = hdul[0].header['Date-OBS']
year= int(hdul[0].header['Date-OBS'].split('-')[0])
day = dayOfYear(date)
time = hdul[0].header['UTC']
hour,minute,sec = map(float,time.split(":"))
metadata = hdul[0].header['METADATA'].split(',')
for element in metadata:
    if "motor_az" in element:
        az = float(element.split(':')[-1])
    elif "motor_el" in element:
        el = float(element.split(':')[-1])
v_lsr = calc_vlsr(year,day,hour,minute,sec,az,el,lat,lon)

#use doppler shift to calculate relative velocity for each frequency bin
lightspeed = 299792
center_freq = 1420.4
velocities = [((center_freq-f)/center_freq)*lightspeed - v_lsr for f in srtdata['frequency']]
srtdata['velocity'] = velocities

In [ ]:
#Plot Corrected Spectrogram- Velocity
samps = np.row_stack([i.data for i in hdul])
vel_mat,s_mat = np.meshgrid(srtdata['velocity'],np.arange(len(hdul)))
print(vel_mat.shape,s_mat.shape,np.fft.fftshift(samps,axes=1).shape)
plt.pcolormesh(s_mat, vel_mat, np.fft.fftshift(samps,axes=1))
cb = plt.colorbar()
plt.ylabel('Velocity (km/s))')
plt.xlabel('Sample Number')
plt.title("Spectrogram")
cb.set_label("power")

[More in-depth version is mapping Galactic rotation curve lab]